In [66]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [67]:
master = gpd.read_file('../export/master/master_acc_bike_pop_street.geojson')

In [68]:
#master.groupby('BZNR').count().sort_values(by='BFSNR', ascending=False)

In [69]:
def percentage(df_in):
    df = df_in.copy()
    
    # Change types etc.
    df = df.replace('...', 0)
    df = df.fillna(0)
    df['YES_IN_PER'] = df['YES_IN_PER'].astype(float)
    df['BIKE_STREET_PER'] = df['BIKE_STREET_PER'].astype(float)

    # Change percentages to decimals
    df['YES_IN_PER'] = df['YES_IN_PER'] / 100
    df['BIKE_STREET_PER'] = df['BIKE_STREET_PER'] / 100

    return df

In [70]:
def calc_ratio(df_in):
    df = df_in
    
    max = df['BIKE_STREET_PER'].max()
    df['MAX_RATIO'] = ((df['BIKE_STREET_PER'] * 100) / max)
    

    return df

In [71]:
def calc_corr_bz(df_in, master):
    df = df_in
    
    #df = df.replace('...', 0)
    #df['YES_IN_PER'] = df['YES_IN_PER'].astype(float) /100
    #df['MAX_RATIO'] = df['MAX_RATIO']/100

    correlations = df.groupby('BZNR')['YES_IN_PER', 'MAX_RATIO'].corr()

    df = correlations.reset_index()
    df_selected = df[df.index % 2 == 0]
    # create a new DataFrame with only the 'BZNR' and 'MAX_RATIO' columns
    df_new = df_selected[['BZNR', 'MAX_RATIO']].copy()
    df_new = df_new.rename(columns={'MAX_RATIO': 'CORR'})

    # merge master and de_new on 'BZNR' and only keep the 'CORR' column from df_new


    df_out = master.merge(df_new, on='BZNR', how='left') # only merge the column 'CORR' from df_new

    df_out = df_out.drop(columns=['MAX_RATIO'])

    
    return df_out
    

In [72]:
def calc_corr_ch(df_in):
    df = df_in
    # calculate correlation between 'YES_IN_PER' and 'BIKE_STREET_PER'
    correlation = df.corr(method='pearson', min_periods=1)['YES_IN_PER']['BIKE_STREET_PER']
    
    return correlation

In [73]:
master = percentage(master)

In [74]:
corr = calc_corr_bz(calc_ratio(master), master)

/var/folders/lb/9kd2fxwx6z79bw37hynhftf00000gn/T/ipykernel_62284/1190528316.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  correlations = df.groupby('BZNR')['YES_IN_PER', 'MAX_RATIO'].corr()


In [75]:
corr

,BFSNR,GMDNAME,ELIGIBLE_VOTERS,TOT_VOTES,PART_PERCENT,VALID_VOTES,YES,NO,YES_IN_PER,BZNR,...,SUM_FREQ,SUM_LENGTH,POP_TOTAL,POP_CH_TOTAL,POP_FOREIGN_TOTAL,LENGTH,COUNT,BIKE_STREET_PER,geometry,CORR
0,1,Aeugst am Albis,1402,664,47.36,663,481,182,0.7255,101,...,53.0,6013.753244,1982.0,1707.0,275.0,89227.171227,725.0,0.06740,"POLYGON ((2678219.000 1235219.000, 2678439.000...",-0.572963
1,2,Affoltern am Albis,7186,2880,40.08,2850,2079,771,0.7295,101,...,57.0,5734.993989,12229.0,8728.0,3501.0,146243.904244,1541.0,0.03922,"POLYGON ((2678219.000 1235219.000, 2677940.000...",-0.572963
2,3,Bonstetten,3656,1701,46.53,1691,1251,440,0.7398,101,...,26.0,3433.248578,5548.0,4660.0,888.0,90191.792826,841.0,0.03807,"POLYGON ((2675803.000 1241039.000, 2675748.000...",-0.572963
3,4,Hausen am Albis,2505,1145,45.71,1132,802,330,0.7085,101,...,95.0,10765.600460,3701.0,3123.0,578.0,144853.747898,1114.0,0.07432,"POLYGON ((2686324.000 1230388.000, 2686024.000...",-0.572963
4,5,Hedingen,2515,1183,47.04,1178,876,302,0.7436,101,...,21.0,4223.776944,3734.0,3142.0,592.0,82904.865295,758.0,0.05095,"POLYGON ((2675803.000 1241039.000, 2676538.000...",-0.572963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2130,6806,Vendlincourt,472,139,29.45,134,97,37,0.7239,2603,...,0.0,0.000000,540.0,502.0,38.0,53832.029439,360.0,0.00000,"POLYGON ((2580412.000 1257077.000, 2580367.000...",0.205687
2131,6807,Basse-Allaine,951,264,27.76,260,191,69,0.7346,2603,...,51.0,7694.840910,1230.0,1129.0,101.0,160638.777810,882.0,0.04790,"POLYGON ((2567009.521 1256868.290, 2566957.590...",0.205687
2132,6808,Clos du Doubs,1096,338,30.84,329,270,59,0.8207,2603,...,92.0,17408.701940,1286.0,1179.0,107.0,387689.703888,1427.0,0.04490,"POLYGON ((2569410.484 1246416.863, 2571130.000...",0.205687
2133,6809,Haute-Ajoie,983,318,32.35,314,224,90,0.7134,2603,...,70.0,11696.744920,1095.0,1011.0,84.0,213324.894625,962.0,0.05483,"POLYGON ((2559718.024 1248149.921, 2560200.000...",0.205687


In [76]:
calc_corr_ch(corr)

/var/folders/lb/9kd2fxwx6z79bw37hynhftf00000gn/T/ipykernel_62284/1683115305.py:4: FutureWarning: The default value of numeric_only in GeoDataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr(method='pearson', min_periods=1)['YES_IN_PER']['BIKE_STREET_PER']


0.06199858610606

In [56]:
corr.to_file('../tmp/corr.shp', driver='ESRI Shapefile')

/var/folders/lb/9kd2fxwx6z79bw37hynhftf00000gn/T/ipykernel_62284/2728858944.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  corr.to_file('../tmp/corr.shp', driver='ESRI Shapefile')


In [77]:
corr.sort_values(by=['CORR'], ascending=True).head(-1)

,BFSNR,GMDNAME,ELIGIBLE_VOTERS,TOT_VOTES,PART_PERCENT,VALID_VOTES,YES,NO,YES_IN_PER,BZNR,...,SUM_FREQ,SUM_LENGTH,POP_TOTAL,POP_CH_TOTAL,POP_FOREIGN_TOTAL,LENGTH,COUNT,BIKE_STREET_PER,geometry,CORR
1111,3551,Brusio,890,351,39.44,349,282,67,0.8080,1842,...,1.0,163.450589,1113.0,967.0,146.0,2.065112e+05,1013.0,0.00079,"POLYGON ((2807791.280 1122837.531, 2807502.625...",-1.000000
1112,3561,Poschiavo,2846,1785,62.72,1708,1335,373,0.7816,1842,...,3.0,1233.754051,3516.0,3225.0,291.0,6.694675e+05,3356.0,0.00184,"POLYGON ((2807951.181 1131712.479, 2807900.966...",-1.000000
1492,5050,Serravalle,1852,703,37.96,675,510,165,0.7556,2102,...,40.0,4466.646651,2071.0,1881.0,190.0,3.411643e+05,1922.0,0.01309,"POLYGON ((2715513.000 1138585.000, 2714758.000...",-0.743024
1491,5049,Blenio,1459,513,35.16,493,321,172,0.6511,2102,...,84.0,17562.286130,1803.0,1661.0,142.0,5.006374e+05,2414.0,0.03508,"POLYGON ((2719305.000 1147844.000, 2718668.000...",-0.743024
1490,5048,Acquarossa,1455,519,35.67,500,351,149,0.7020,2102,...,131.0,13255.677600,1808.0,1590.0,218.0,3.174076e+05,1844.0,0.04176,"POLYGON ((2719305.000 1147844.000, 2719670.000...",-0.743024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,261,Zürich,230556,92241,40.01,91491,72263,19228,0.7898,112,...,1635.0,107483.106100,415367.0,280565.0,134802.0,1.633215e+06,22874.0,0.06581,"POLYGON ((2682965.000 1242721.000, 2682728.000...",NaN
496,1061,Luzern,53034,20376,38.42,20163,14858,5305,0.7369,311,...,356.0,28737.648710,81691.0,61927.0,19764.0,4.641357e+05,6093.0,0.06192,"MULTIPOLYGON (((2675115.000 1206205.000, 26750...",NaN
595,1301,Einsiedeln,11016,4205,38.17,4192,2731,1461,0.6515,501,...,320.0,37579.720580,15870.0,13311.0,2559.0,5.962595e+05,3719.0,0.06303,"POLYGON ((2702735.000 1225625.000, 2703170.000...",NaN
596,1311,Gersau,1555,645,41.48,644,377,267,0.5854,502,...,34.0,6041.185703,2319.0,1764.0,555.0,1.285967e+05,831.0,0.04698,"POLYGON ((2678725.000 1205476.000, 2680769.000...",NaN


In [ ]:
def calc_bike_street_ratio(df_in):
    df = df_in.copy()
    df['BIKE_STREET_PER'] = np.round((df['SUM_LENGTH'] / df['LENGTH']) , 2)

    return df

In [ ]:
def calc_ratio(df_in):
    df = df_in.copy()
    #replace entries with '...' with 0
    df['YES_IN_PER'] = df['YES_IN_PER'].replace('...', 0)
    #change type of 'YES_IN_PER' to float
    df['YES_IN_PER'] = df['YES_IN_PER'].astype(float)
    #fill na values with 0
    df['YES_IN_PER'] = df['YES_IN_PER'].fillna(0)
    #Fill na values with 0 for all columns
    df = df.fillna(0)
    #df['DIV'] = (df['BIKE_STREET_PER'] / df['YES_IN_PER'])
    #df['ADD'] = (df['BIKE_STREET_PER'] + df['YES_IN_PER'])
    #df['SUB'] = (df['BIKE_STREET_PER'] - df['YES_IN_PER'])
    #df['MUL'] = (df['BIKE_STREET_PER']/100) * (df['YES_IN_PER']/100)
    ## PER DIFF subtract value A from value B, divide the result by the average of the two values, and then multiply by 100 to get a percentage
    #df['PER_DIFF_Y'] = (df['YES_IN_PER'] - df['BIKE_STREET_PER']) / ((df['BIKE_STREET_PER'] + df['YES_IN_PER'])/2) * 100
    #df['PER_DIFF_B'] = (df['BIKE_STREET_PER'] - df['YES_IN_PER']) / ((df['BIKE_STREET_PER'] + df['YES_IN_PER'])/2) * 100


    return df